Install dependancies

In [ ]:
%pip install EbookLib
%pip install ebooklib
%pip install requests
%pip install ollama
%pip install gradio_client
%pip install websocket-client
%pip install Pillow

Define things

In [2]:
import time
import re
import os
from ebooklib import epub
import base64
import requests
import json
import ollama


def remove_first_line(test_string):
    if test_string.startswith("Here") and test_string.split("\n")[0].strip().endswith(":"):
        return re.sub(r'^.*\n', '', test_string, count=1)
    return test_string

def generate_text(prompt, model="llama3", max_tokens=2000, temperature=0.7):
    response = ollama.chat(
        model=model,
        options={'temperature': temperature},
        messages=[
        {
            'role': 'system',
            'content': 'You are a world-class author. Write the requested content with great skill and attention to detail.',
        },
        {
            'role': 'user',
            'content': prompt,
        },
    ])
    response_text = response['message']['content']
    return response_text.strip()

def generate_cover_prompt(plot):
    response = generate_text(f"""Plot: {plot}
    --
    Describe the cover we should create, based on the plot. This should be two sentences long, maximum. The prompt must be styled as a stable diffusion prompt.
    --
    Example 1:
    portrait of a pretty blonde woman, a flower crown, earthy makeup, flowing maxi dress with colorful patterns and fringe, a sunset or nature scene, green and gold color scheme
    
    Example 2:
    photorealistic, visionary portrait of a dignified older man with weather-worn features, digitally enhanced, high contrast, chiaroscuro lighting technique, intimate, close-up, detailed, steady gaze, rendered in sepia tones, evoking rembrandt, timeless, expressive, highly detailed, sharp focus, high resolution
    
    Example 3:
    a living room, bright modern Scandinavian style house, large windows, magazine photoshoot, 8k, studio lighting
    
    Example 4:
    close up photo of a rabbit, forest in spring, haze, halation, bloom, dramatic atmosphere, centred, rule of thirds, 200mm 1.4f macro shot
    
    Example 5:
    a glamorous digital magazine photoshoot, a fashionable model wearing avant-garde clothing, set in a futuristic cyberpunk roof-top environment, with a neon-lit city background, intricate high fashion details, backlit by vibrant city glow, Vogue fashion photography
    
    Example 6:
    long exposure photo of tokyo street, blurred motion, streaks of light, surreal, dreamy, ghosting effect, highly detailed
    
    Example 7:
    double exposure portrait of a beautiful woman with brown hair and a snowy tree under the bright moonlight by Dave White, Conrad Roset, Brandon Kidwell, Andreas Lie, Dan Mountford, Agnes Cecile, splash art, winter colours, gouache, triadic colours, thick opaque strokes, brocade, depth of field, hyperdetailed, whimsimcal, amazing depth, dynamic, dreamy masterwork
    """)
    return response

def generate_title(plot):
    print("Generating title...")
    response = generate_text(f"Here is the plot for the book: {plot}\n\n--\n\nRespond with a great title for this book. Only respond with the title, nothing else is allowed.")
    print("Title generated.")
    return remove_first_line(response)
       
def create_cover_image(plot):
    print("Generating cover image...")
    cover_prompt = str(generate_cover_prompt(plot))
    
    # Define the URL and the payload to send.
    url = "http://127.0.0.1:7860"
    
    payload = {
    "prompt": cover_prompt,
    "steps": 20,
    "negative_prompt": "lowres, text, error, cropped, worst quality, low quality, jpeg artifacts, ugly, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, blurry, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, fused fingers, too many fingers, long neck, username, watermark, signature,NSFW, naked, nude"
    }

    # Send said payload to said URL through the API.
    response = requests.post(url=f'{url}/sdapi/v1/txt2img', json=payload)
    r = response.json()

    # Decode and save the image.
    with open("./cover.png", 'wb') as f:
        f.write(base64.b64decode(r['images'][0]))
    print("Cover image generated.")

def generate_chapter_title(chapter_content):
    response = generate_text(f"Chapter Content:\n\n{chapter_content}\n\n--\n\nGenerate a concise and engaging title for this chapter based on its content. Respond with the title only, nothing else.")
    return remove_first_line(response)

def create_epub(title, author, chapters, cover_image_path='cover.png'):
    print("Creating the EPUB file...")
    book = epub.EpubBook()
    # Set metadata
    book.set_identifier('id123456')
    book.set_title(title)
    book.set_language('en')
    book.add_author(author)
    # Add cover image
    with open(cover_image_path, 'rb') as cover_file:
        cover_image = cover_file.read()
    book.set_cover('cover.png', cover_image)
    # Create chapters and add them to the book
    epub_chapters = []
    for i, chapter_content in enumerate(chapters):
        chapter_title = generate_chapter_title(chapter_content)
        chapter_file_name = f'chapter_{i+1}.xhtml'
        epub_chapter = epub.EpubHtml(title=chapter_title, file_name=chapter_file_name, lang='en')
        # Add paragraph breaks
        formatted_content = ''.join(f'<p>{paragraph.strip()}</p>' for paragraph in chapter_content.split('\n') if paragraph.strip())
        epub_chapter.content = f'<h1>{chapter_title}</h1>{formatted_content}'
        book.add_item(epub_chapter)
        epub_chapters.append(epub_chapter)


    # Define Table of Contents
    book.toc = (epub_chapters)

    # Add default NCX and Nav files
    book.add_item(epub.EpubNcx())
    book.add_item(epub.EpubNav())

    # Define CSS style
    style = '''
    @namespace epub "http://www.idpf.org/2007/ops";
    body {
        font-family: Cambria, Liberation Serif, serif;
    }
    h1 {
        text-align: left;
        text-transform: uppercase;
        font-weight: 200;
    }
    '''

    # Add CSS file
    nav_css = epub.EpubItem(uid="style_nav", file_name="style/nav.css", media_type="text/css", content=style)
    book.add_item(nav_css)

    # Create spine
    book.spine = ['nav'] + epub_chapters

    # Save the EPUB file
    epub.write_epub(f'{title}.epub', book)
    print(f"EPUB file '{title}.epub' created.")


def generate_book(writing_style, book_description, num_chapters):
    print("Generating plot outline...")
    plot_prompt = f"Create a detailed plot outline for a {num_chapters}-chapter book in the {writing_style} style, based on the following description:\n\n{book_description}\n\nEach chapter should be at least 10 pages long."
    plot_outline = generate_text(plot_prompt)
    print("Plot outline generated.")

    chapters = []
    for i in range(num_chapters):
        print(f"Generating chapter {i+1}...")
        chapter_prompt = f"Previous Chapters:\n\n{' '.join(chapters)}\n\nWriting style: `{writing_style}`\n\nPlot Outline:\n\n{plot_outline}\n\nWrite chapter {i+1} of the book, ensuring it follows the plot outline and builds upon the previous chapters. The chapter should be at least 256 paragraphs long... we're going for lengthy yet exciting chapters here."
        chapter = generate_text(chapter_prompt, max_tokens=4000)
        chapters.append(remove_first_line(chapter))
        print(f"Chapter {i+1} generated.")
        time.sleep(1)  # Add a short delay to avoid hitting rate limits

    print("Compiling the book...")
    book = "\n\n".join(chapters)
    print("Book generated!")

    return plot_outline, book, chapters

Execute it. Make sure Ollama is serving llama3, make sure AUTOMATIC1111 is running with the --api flag.

In [ ]:
# User input
writing_style = input("Enter the desired writing style: ")
book_description = input("Enter a high-level description of the book: ")
num_chapters = int(input("Enter the number of chapters: "))

# Generate the book
plot_outline, book, chapters = generate_book(writing_style, book_description, num_chapters)

title = generate_title(plot_outline)

# Make sure the title is windows compatible
# In one line
title = re.sub(r'[<>:"/\\|?*]', '', title)


# Save the book to a file
print("Saving the txt book...")
with open(f"{title}.txt", "w") as file:
    file.write(book)
print(f"Book saved as a {title}.txt.")

create_cover_image(plot_outline)

# Create the EPUB file
create_epub(title, 'AI', chapters, './cover.png')
